In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import spacy
import json
import random
import re

from spacy import displacy

In [10]:
#function to load the data

def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

In [11]:
#function to save the data

def save_data(file, data):
    with open(file, "w", encoding="uft-8") as f:
        json.dump(data, f, indent=4)

In [12]:
#function to train the data

def train_spacy(data, iterations):
    TRAIN_DATA = data

    #loading the blank model to train
    nlp = spacy.blank("en")

    #adding ner pipe to loaded model
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)

    #adding label to the model in pipe
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    
    #disabling other pipes to model
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):

      #training begin to the model
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print ("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                            [text],
                            [annotations],
                            drop=0.2,
                            sgd=optimizer,
                            losses=losses
                )
            print (losses)
    return (nlp)

In [13]:
#function to clean the data

def clean_text(text):
  cleaned = re.sub(r"[\(\[].*?[\)\]]", "",text)
  return (cleaned)

In [14]:
#read the data from json file

TRAIN_DATA = load_data("/content/drive/MyDrive/Phase1/common_noun_d1.json")

FileNotFoundError: ignored

In [ ]:
TRAIN_DATA

In [ ]:
#call the train function with iterations

nlp = train_spacy(TRAIN_DATA, 30)
nlp.to_disk("/content/drive/MyDrive/Phase1/common_noun_model")

In [ ]:
nlp = spacy.load('/content/drive/MyDrive/Phase1/common_noun_model')
nlp.entity.cfg

In [ ]:
#reading the fresh data

test = "John is a good boy , he goes to school everyday and gains knowledge, he has a friend named Rahul in the class and they were like a family. Today Rahul was hungry and they both went to Johns home to eat , after having their lunch they both played sports team game like cricket , golf, tennis , etc. The next day, they went to a circus where they say all of my  horse race"

In [ ]:
#clean_text calls the clean function and removes all unwanted symbols

test = clean_text(test)

In [ ]:
#loading the created model

nlp = spacy.load("/content/drive/MyDrive/Phase1/common_noun_model")

In [ ]:
doc = nlp(test)

In [ ]:
#printing the entities

for ent in doc.ents:
  print (ent.text, ent.label_)

In [ ]:
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
#entity_frame = spacy.DataFrame(named_entities, 
#                            columns=['Entity Name', 'Entity Type'])
#
#top_entities = (entity_frame.groupby(by=['Entity Name', 'Entity Type'])
#                           .size()
#                           .sort_values(ascending=False)
#                           .reset_index().rename(columns={0 : 'Frequency'}))
#top_entities.T.iloc[:,:15]

In [ ]:


test1 = "KLE Techniological University has a committee of trained lecturers where Dr.Vishwanath sir is PG coordinator and Dr.Karibassappa sir is head of examination department and  Dr.Tothad sir had a conference in International University of France"

In [ ]:
#clean_text calls the clean function and removes all unwanted symbols

test1 = clean_text(test1)

In [ ]:
#loading the created model

nlp = spacy.load("/content/drive/MyDrive/Phase1/common_noun_model")

In [ ]:
doc = nlp(test1)

In [ ]:
#printing the entities

for ent in doc.ents:
  print (ent.text, ent.label_)

In [ ]:
displacy.render(doc, style='ent', jupyter=True)

In [ ]:
from spacy.gold import GoldParse
from spacy.scorer import Scorer

In [ ]:
#
#def evaluate(ner_model, TRAIN_DATA):
#    scorer = Scorer()
#    for input_, annot in TRAIN_DATA:
#        doc_gold_text = ner_model.make_doc(input_)
#        gold = GoldParse(doc_gold_text, entities=annot)
#        pred_value = ner_model(input_)
#        scorer.score(pred_value, gold)
#    return scorer.scores

In [ ]:
#ner_model = spacy.load("/content/drive/MyDrive/Noun-json/nlp_model") 
#results = evaluate(ner_model, TRAIN_DATA)

In [ ]:
#results

In [ ]:
import random
import logging
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score
import re
import spacy
from sklearn import metrics

In [ ]:

def trim_entity_spans(data: test) -> list:
    invalid_span_tokens = re.compile(r'\s')
    cleaned_data = []
    for text, annotations in TRAIN_DATA:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data


In [ ]:

def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, encoding="utf-8") as f:
            lines = f.readlines()            
        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))
            training_data.append((text, {"entities" : entities}))
        return training_data
    except Exception as e:
        #logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None



In [ ]:
def train_spacy():
    TRAIN_DATA = convert_dataturks_to_spacy("/content/drive/MyDrive/Phase1/common_noun_d1.json")
    TRAIN_DATA = trim_entity_spans(TRAIN_DATA)
    examples = convert_dataturks_to_spacy("/content/drive/MyDrive/Phase1/common_noun_d1.json")
    examples = trim_entity_spans(examples)
    tp=0
    tr=0
    tf=0
    ta=0
    c=0        
    for text,annot in examples:
        f=test
        nlp_trained = spacy.load("/content/drive/MyDrive/Phase1/common_noun_model")
        doc_to_test=nlp_trained(text)
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[]
        for ent in doc_to_test.ents:
            d[ent.label_].append(ent.text)
        #for i in set(d.keys()):
         #   f.write("\n\n")
          #  f.write(i +":"+"\n")
           # for j in set(d[i]):
            #    f.write(j.replace('\n','')+"\n")
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[0,0,0,0,0,0]
        for ent in doc_to_test.ents:
            doc_gold_text= nlp_trained.make_doc(text)
            gold = GoldParse(doc_gold_text, entities=annot.get("entities"))
            y_true = [ent.label_ if ent.label_ in x else 'Not '+ent.label_ for x in gold.ner]
            y_pred = [x.ent_type_ if x.ent_type_ ==ent.label_ else 'Not '+ent.label_ for x in doc_to_test]
            #print(metrics.confusion_matrix(y_true, y_pred))
            #print(metrics.classification_report(y_true, y_pred))
            if(d[ent.label_][0]==0):
                (p,r,f,s)= precision_recall_fscore_support(y_true,y_pred,average='weighted')
                a=accuracy_score(y_true,y_pred)
                d[ent.label_][0]=1
                d[ent.label_][1]+=p
                d[ent.label_][2]+=r
                d[ent.label_][3]+=f
                d[ent.label_][4]+=a
                d[ent.label_][5]+=1
           
        c+=1
    #cnf_matrix = confusion_matrix(y_true,y_pred)
    #disp = ConfusionMatrixDisplay(confusion_matrix=cnf_matrix,display_labels=ent.label_)
    #fig, ax = plt.subplots(figsize=(8,8))
    #disp = disp.plot(include_values=True,cmap='Blues', xticks_rotation='vertical', values_format='.3g', ax=ax)
    #accuracy1=(cnf_matrix[0,0]+cnf_matrix[1,1])/total1
    #print ('Accuracy : ', accuracy1)
    #plt.title('Training and test loss')
    #plt.show()   

    for i in d:
        print("\n For Entity "+i+"\n")
        print("Accuracy : "+str((d[i][4]/d[i][5])*100)+"%")
        print("Precision : "+str(d[i][1]/d[i][5]))
        print("Recall : "+str(d[i][2]/d[i][5]))
        print("F-score : "+str(d[i][3]/d[i][5]))


In [ ]:
train_spacy()